In [1]:
import gzip 
from lxml import etree
import pandas as pd 
import xmltodict
import glob
import os 
import time 
import polars as pl 
import duckdb
import shutil 


In [2]:
DOSSIER_500 = "./todo/test1000_2/"
DOSSIER_TODO = './fichiers/todo_xml/'
DOSSIER_PARQUET_UNITAIRE = './fichiers/todo_parquet/'

In [6]:
def timtim(fonction):
    def wrapper(*args, **kwargs):
        debut = time.time()
        resultat = fonction(*args, **kwargs)
        fin = time.time()
        temps_execution = fin - debut
        print(f"La fonction {fonction.__name__} a pris {temps_execution:.5f} secondes pour s'exécuter.")
        return resultat
    return wrapper

In [7]:
def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

In [12]:
@timtim
def Methode_1(chemin) : 
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   liste_methode_1 = []
   for ligne_methode_1 in docbase:
     dict_temp = {}
     for clef, valolo in ligne_methode_1.items():
         if isinstance(valolo, dict) and '@V' in valolo:
             val_atrib = valolo['@V']
         else:
             val_atrib = valolo
         dict_temp[clef] = val_atrib
     liste_methode_1.append(dict_temp)
   df_methode_1 = pd.DataFrame(liste_methode_1)
   liste_df.append(df_methode_1)
  except Exception as e :
   print(f'Erreur fichier {fichier}') 
   continue 
 
 df_gros_1 = pd.concat(liste_df)
 return df_gros_1

@timtim
def Methode_2(chemin) : 
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   df_methode_2 = pd.DataFrame(docbase)
   nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
   for col in df_methode_2.columns : 
    if col in ['MtSup', 'CaracSup'] : 
     continue 
    else :
     df_methode_2[col] = df_methode_2[col].apply(nettoyage)
   #colonnes_a_exclure = ['MtSup', 'CaracSup']
   #nettoyage = lambda x: str(x).replace("{'@V': '", "").replace("'}", "") if x.name not in colonnes_a_exclure else x
   #df_methode_2 = df_methode_2.apply(nettoyage)
   liste_df.append(df_methode_2)
  except Exception as e : 
    print(f'Erreur fichier {fichier}')
    continue
   
 gros_df_2 = pd.concat(liste_df)
 return gros_df_2

@timtim
def Methode_2_test(chemin) : 
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   df_methode_2 = pd.DataFrame(docbase)
   #colonnes_a_exclure = ['MtSup', 'CaracSup']
   #nettoyage = lambda x: str(x).replace("{'@V': '", "").replace("'}", "") if x.name not in colonnes_a_exclure else x
   #df_methode_2 = df_methode_2.apply(nettoyage)
   liste_df.append(df_methode_2)
  except Exception as e : 
    print(f'Erreur fichier {fichier}')
    continue  

 gros_df_2 = pd.concat(liste_df)
 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in gros_df_2.columns : 
    if col in ['MtSup', 'CaracSup'] : 
     continue 
    else :
     gros_df_2[col] = gros_df_2[col].apply(nettoyage)
 gros_df_2 = gros_df_2.reset_index(drop=True)
 return gros_df_2 

@timtim
def Methode_3(chemin) : 
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   df_methode_3 = pd.json_normalize(docbase)
   liste_df.append(df_methode_3)
  except Exception as e : 
    print(f'Erreur fichier {fichier}')
    continue
 gros_df_3 = pd.concat(liste_df)
 return gros_df_3

SCHEMA = {"Nature" : pl.Object,
          "LibCpte" : pl.Object,
          "Fonction" : pl.Object,
          "Operation" : pl.Object,
          "ContNat" : pl.Object,
          "ArtSpe" : pl.Object,
          "ContFon" : pl.Object,
          "ContOp" : pl.Object,
          "CodRD" : pl.Object,
          "MtBudgPrec" : pl.Object,
          "MtRARPrec" : pl.Object,
          "MtPropNouv" : pl.Object,
          "MtPrev" : pl.Object,
          "CredOuv" : pl.Object,
          "MtReal" : pl.Object,
          "MtRAR3112" : pl.Object,
          "OpBudg" : pl.Object,
          "TypOpBudg" : pl.Object,
          "OpeCpteTiers" : pl.Object, 
          "MtSup" : pl.Object, 
          "CaracSup" : pl.Object
          }

@timtim
def Methode_polars(chemin, schema) : 
 liste_prepl = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   dico = parse_fichier(fichier)
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   pre_df = pl.DataFrame(docbase, schema=schema)
   liste_prepl.append(pre_df)
  except Exception as e : 
    print(f'Erreur fichier {fichier}')
    continue
 pl_test = pl.concat(liste_prepl)
 return pl_test 

In [22]:
dfm1 = Methode_1(DOSSIER_TODO)

Erreur fichier ./todo\619027.xml.gz
Erreur fichier ./todo\628857.xml.gz
Erreur fichier ./todo\682109.xml.gz
Erreur fichier ./todo\722795.xml.gz
Erreur fichier ./todo\730073.xml.gz
Erreur fichier ./todo\748147.xml.gz
Erreur fichier ./todo\756300.xml.gz
Erreur fichier ./todo\775775.xml.gz
Erreur fichier ./todo\807134.xml.gz
Erreur fichier ./todo\809162.xml.gz
Erreur fichier ./todo\809290.xml.gz
Erreur fichier ./todo\819789.xml.gz
Erreur fichier ./todo\821461.xml.gz
Erreur fichier ./todo\838663.xml.gz
La fonction Methode_1 a pris 341.08701 secondes pour s'exécuter.


In [130]:
dfm2 = Methode_2(DOSSIER_TODO)

Erreur fichier ./todo\775775.xml.gz
La fonction Methode_2 a pris 764.44217 secondes pour s'exécuter.


In [30]:
dfm2.columns

Index(['Nature', 'ContNat', 'ArtSpe', 'CodRD', 'MtBudgPrec', 'MtPropNouv',
       'MtPrev', 'CredOuv', 'OpBudg', 'MtSup', 'CaracSup', 'TypOpBudg',
       'MtRARPrec', 'LibCpte', 'Operation', 'ContOp', 'MtReal', 'MtRAR3112',
       'Fonction', 'OpeCpteTiers', 'ContFon'],
      dtype='object')

In [24]:
dfm3 = Methode_3(DOSSIER_TODO)

Erreur fichier ./todo\775775.xml.gz
La fonction Methode_3 a pris 386.22054 secondes pour s'exécuter.


In [15]:
dfm21 = Methode_2_test(DOSSIER_TODO)

Erreur fichier ./fichiers/todo_xml\775775.xml.gz
La fonction Methode_2_test a pris 695.09354 secondes pour s'exécuter.


In [34]:
dfm3.columns

Index(['Nature.@V', 'ContNat.@V', 'ArtSpe.@V', 'CodRD.@V', 'MtBudgPrec.@V',
       'MtPropNouv.@V', 'MtPrev.@V', 'CredOuv.@V', 'OpBudg.@V', 'MtSup.@V',
       'MtSup.@Code', 'CaracSup.@V', 'CaracSup.@Code', 'TypOpBudg.@V', 'MtSup',
       'MtRARPrec.@V', 'CaracSup', 'LibCpte.@V', 'Operation.@V', 'ContOp.@V',
       'MtReal.@V', 'MtRAR3112.@V', 'Fonction.@V', 'OpeCpteTiers.@V',
       'ContFon.@V'],
      dtype='object')

In [ ]:
comparatif_pl = Methode_polars(DOSSIER_TODO, schema=SCHEMA)

In [36]:
comparatif_pl.columns

['Nature',
 'LibCpte',
 'Fonction',
 'Operation',
 'ContNat',
 'ArtSpe',
 'ContFon',
 'ContOp',
 'CodRD',
 'MtBudgPrec',
 'MtRARPrec',
 'MtPropNouv',
 'MtPrev',
 'CredOuv',
 'MtReal',
 'MtRAR3112',
 'OpBudg',
 'TypOpBudg',
 'OpeCpteTiers',
 'MtSup',
 'CaracSup']

Outpout : 

**Methode_1**, extraction depuis les dictionnaires : 

- Erreur fichier ./todo\619027.xml.gz
- Erreur fichier ./todo\628857.xml.gz
- Erreur fichier ./todo\682109.xml.gz
- Erreur fichier ./todo\722795.xml.gz
- Erreur fichier ./todo\730073.xml.gz
- Erreur fichier ./todo\748147.xml.gz
- Erreur fichier ./todo\756300.xml.gz
- Erreur fichier ./todo\775775.xml.gz
- Erreur fichier ./todo\807134.xml.gz
- Erreur fichier ./todo\809162.xml.gz
- Erreur fichier ./todo\809290.xml.gz
- Erreur fichier ./todo\819789.xml.gz
- Erreur fichier ./todo\821461.xml.gz
- Erreur fichier ./todo\838663.xml.gz

- La fonction Methode_1 a pris **341.08701** secondes pour s'exécuter.



**Methode_2**, DataFrame : 

- Erreur fichier ./todo\775775.xml.gz

- La fonction Methode_2 a pris **414.56070** secondes pour s'exécuter.


**Methode_3**, json_normalize : 

- Erreur fichier ./todo\775775.xml.gz

- La fonction Methode_3 a pris **386.22054** secondes pour s'exécuter.


**Methode Polars** : 

- Erreur fichier ./todo\619027.xml.gz
- Erreur fichier ./todo\628857.xml.gz
- Erreur fichier ./todo\682109.xml.gz
- Erreur fichier ./todo\722795.xml.gz
- Erreur fichier ./todo\730073.xml.gz
- Erreur fichier ./todo\748147.xml.gz
- Erreur fichier ./todo\756300.xml.gz
- Erreur fichier ./todo\775775.xml.gz
- Erreur fichier ./todo\807134.xml.gz
- Erreur fichier ./todo\809162.xml.gz
- Erreur fichier ./todo\809290.xml.gz
- Erreur fichier ./todo\819789.xml.gz
- Erreur fichier ./todo\821461.xml.gz
- Erreur fichier ./todo\838663.xml.gz

- La fonction Methode_polars a pris **366.51400** secondes pour s'exécuter.


### Resultats sur 20 000 fichiers, pour l'extract des lignes budgetaires : 

- Methode extract depuis dict : 341.08701 sec, 14 erreurs 
- Methode DataFrame : 414.56070 sec, 1 erreur
- Methode json : 386.22054 sec, 1 erreur
- Methode Polars : 366.51400 sec, 14 erreurs 

Une erreure commune parmi les 4 méthode, sur le fichier 775775, il s'avère qu'il est impossible à ouvrir car l'archive est endommagée. 

La méthode Json est plus rapide sur le traitement, notamment car elle n'a pas besoin de nettoyer les @V dans les données, les @V n'apparaissent que sur les libelles des colonnes, cependant la méthode json_normalize traite les Carac Sup et Mt Sup de façon particulière, les rendants inexploitables. Ce gain de temps sera donc perdu lorsque l'on voudra utiliser ces informations. 

La méthode DataFrame elle, considère Mtsup et CaracSup comme des colonnes composées de listes de dictionnaires / de dictionnaires, les rendant exploitables plus tard. 

# MtSup et CaracSup

In [ ]:
dft = dfm2.copy()

In [ ]:
dft.head(5)

In [126]:
nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
for col in dft.columns : 
 if col in ['MtSup', 'CaracSup'] : 
  continue 
 else :
  dft[col] = dft[col].apply(nettoyage)

In [ ]:
dft.head(5)

In [88]:
dft2 = dfm2.copy()

In [ ]:
dft2.head(1)

In [48]:
colonnes_a_exclure = ['MtSup', 'CaracSup']

# Fonction de nettoyage
def nettoyage(colonne):
    if colonne.name not in colonnes_a_exclure:
        return colonne.apply(lambda x: str(x).replace("{'@V': '", "").replace("'}", ""))
    else:
        return colonne

# Appliquer la fonction de nettoyage à chaque colonne du DataFrame
dft2 = dft2.apply(nettoyage)

In [ ]:
dft2['MtSup'] = dft2['MtSup'].apply(lambda x: str(x))
dft2['CaracSup'] = dft2['CaracSup'].apply(lambda x: str(x))
dft2.head(5)

In [66]:
dft2['MtSup'] = dft2['MtSup'].astype(str)
dft2['CaracSup'] = dft2['CaracSup'].astype(str)
dft2['MtSup'].dtypes

dtype('O')

In [ ]:
dft3 = dfm2[['Nature', 'Fonction']]
dft3

In [107]:
nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
for col in dft3.columns : 
 dft3[col] = dft3[col].apply(nettoyage)
dft3

dft3['Nature'].dtypes

C:\Users\v_enz\AppData\Local\Temp\ipykernel_11740\254260435.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft3[col] = dft3[col].apply(nettoyage)


dtype('O')

In [112]:
dft3.columns
dft3_reset_index = dft3.reset_index(drop=True)


In [114]:
dft3_reset_index.to_parquet('./test/renouveau/df_m2_parquet', engine='pyarrow')


In [ ]:
def _transformation_mtsup_test1(lignes_budget: dict) -> dict:
 '''Grâce aux spaghettis, il y a deux types de MtSup dans les fichiers,
       des dict et des listes de dict, permet de gérer les deux cas
 '''
 for i in lignes_budget:
  type_mtsup = i.get('MtSup')  # Permet de connaitre le type de MtSup
  if type_mtsup is not None:  # Vérifie si la clé 'MtSup' existe
   if isinstance(type_mtsup, dict):
    dict_mtsup = type_mtsup
    i['MtSup_1_Lib'] = {'@V': dict_mtsup.get('@Code', '')}
    i['MtSup_1_Val'] = {'@V': dict_mtsup.get('@V', '')}
   elif isinstance(type_mtsup, list):
    dict_mtsup = i['MtSup']
    if isinstance(dict_mtsup, dict):  # Vérifie si c'est un dict dans la liste
     mtsup_propre = {
                        'MtSup_1_Lib': {'@V': dict_mtsup.get('@Code', '')},
                        'MtSup_1_Val': {'@V': dict_mtsup.get('@V', '')}
                    }
    else:
     mtsup_propre = {}
     for z, entry in enumerate(dict_mtsup, start=1):
      code = f'MtSup_{z}_Lib'
      valeur = f'MtSup_{z}_Val'
      mtsup_propre[code] = entry.get('@Code', '')
      mtsup_propre[valeur] = entry.get('@V', '')
      i.update(mtsup_propre)
                
    return lignes_budget
   
def _transformation_mtsup_original(lignes_budget: dict) -> dict:
 '''Grâce aux spaghettis, il y a deux types de MtSup dans les fichiers,
       des dict et des listes de dict, permet de gérer les deux cas
    '''
 for i in lignes_budget:
  type_mtsup = i.get('MtSup')  # Permet de connaitre le type de MtSup
  if type_mtsup is not None:  # Vérifie si la clé 'MtSup' existe
    if isinstance(type_mtsup, dict):
      dict_mtsup = type_mtsup
      i['MtSup_1_Lib'] = {'@V': dict_mtsup.get('@Code', '')}
      i['MtSup_1_Val'] = {'@V': dict_mtsup.get('@V', '')}
    elif isinstance(type_mtsup, list):
      dict_mtsup = i['MtSup']
      mtsup_propre = {}
      for z, entry in enumerate(dict_mtsup, start=1):
        code = f'MtSup_{z}_Lib'
        valeur = f'MtSup_{z}_Val'
        mtsup_propre[code] = entry.get('@Code', '')
        mtsup_propre[valeur] = entry.get('@V', '')
        i.update(mtsup_propre)
        
 return lignes_budget



### Ajout de la verif sur le fichier parquet 

1. Methode Duckdb 

In [19]:
def recherche_id_fichier(chemin_parquet) :
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet(chemin_parquet)
 requete_duckdb = ''' 
 SELECT
    DISTINCT Id_Fichier
 FROM 
    docubudg_t
 '''
 result_requete= conduck.execute(requete_duckdb).fetchdf()
 liste_id = result_requete['Id_Fichier'].to_list()
 conduck.close()
 return liste_id


def _isolement_id(fichier) : 
 '''Extrait l'id du nom du fichier pour la liste comprehension de securité

 ATTENTION, le premier split / va changer si on l'applique sur du minio '''
 val_id_fichier_source = fichier.split("\\")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier

In [50]:
@timtim
def Methode_2_1(chemin, chemin_verif) : 
 liste_id = recherche_id_fichier(chemin_verif)
 liste_df = []
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  id_fichier = _isolement_id(fichier)
  #Fichier non pre existant dans la db / parquet
  if id_fichier in liste_id :
   pass 
  else : 
   try : 
     fichier_parse = parse_fichier(fichier)
     metadonnees = fichier_parse['DocumentBudgetaire']['EnTeteDocBudgetaire']
     donnes_budget = fichier_parse['DocumentBudgetaire']['Budget']['LigneBudget']
     metadonnees['Id_Fichier'] = {'@V' : id_fichier}
     for i in donnes_budget : 
      i.update(metadonnees)
     def_2_1 = pd.DataFrame(donnes_budget)
     nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
     for col in def_2_1.columns : 
      def_2_1[col] = def_2_1[col].apply(nettoyage)
     liste_df.append(def_2_1)
   except Exception as e : 
     print(f'Erreur fichier {fichier}')
     continue
 df_2_1 = pd.concat(liste_df)
 df_2_1.to_parquet('parquet_renouveau', compression= 'gzip')
 return df_2_1 


In [27]:
@timtim
def Extraction_xml_to_parquet(chemin) : 
 ''' Traite les XML et les envoies sous format parquet, pas encore de nettoyage'''
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees['Id_Fichier'] = {'@V' : id_fichier}
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   for i in docbase : 
      i.update(metadonnees)
   df_methode_2 = pd.DataFrame(docbase)
   df_methode_2['MtSup'] = df_methode_2['MtSup'].astype(str)
   df_methode_2['CaracSup'] = df_methode_2['CaracSup'].astype(str)
   df_methode_2 = df_methode_2.reset_index(drop=True)
   df_methode_2.to_parquet(f'./fichiers/todo_parquet/{id_fichier}', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(e)
    break  

In [28]:
chemin_test_xml_500 = './fichiers/todo_xml/test1000/t500/'
chemin_test_xml_solo = './fichiers/todo_xml/test1000/t500/612019.xml.gz'
chemin_test_xml_solo

'./fichiers/todo_xml/test1000/t500/612019.xml.gz'

In [29]:
Extraction_xml_to_parquet(chemin_test_xml_500)

Erreur fichier ./fichiers/todo_xml/test1000/t500\612023.xml.gz, extraction impossible
'MtSup'
La fonction Extraction_xml_to_parquet a pris 0.02500 secondes pour s'exécuter.


In [23]:
for ss in glob.glob(os.path.join(chemin_test_xml_500, "*.gz")) : 
 id_ss = _isolement_id(ss)
 print(id_ss)

612019
612023
612027
612030
612038
612050
612057
612058
612069
612078
612117
612127
612128
612131
612136
612151
612159
612160
612172
612180
612188
612192
612220
612221
612223
612240
612241
612245
612262
612264
612266
612269
612271
612289
612292
612295
612328
612339
612347
612350
612365
612370
612376
612406
612415
612420
612424
612425
612450
612458
612482
612492
612499
612527
612529
612546
612547
612560
612561
612563
612598
612621
612633
612652
612661
612664
612675
612678
612680
612690
612711
612733
612755
612760
612780
612787
612804
612821
612826
612829
612847
612876
612883
612896
612902
612908
612914
612923
612943
612984
612987
612991
612998
613002
613011
613012
613025
613029
613032
613068
613074
613088
613113
613124
613134
613135
613143
613145
613152
613182
613186
613189
613217
613234
613235
613254
613272
613291
613298
613309
613320
613339
613345
613362
613370
613379
613385
613386
613407
613408
613410
613413
613437
613467
613472
613483
613485
613510
613517
613524
613526
613535
613562

In [16]:

id_fichier_b = _isolement_id(chemin_test_xml_solo)
dico_b = parse_fichier(chemin_test_xml_solo)
metadonnees_b = dico_b['DocumentBudgetaire']['EnTeteDocBudgetaire']
metadonnees_b['Id_Fichier'] = {'@V' : id_fichier_b}
docbase_b = dico_b['DocumentBudgetaire']['Budget']['LigneBudget']
for i in docbase_b : 
 i.update(metadonnees_b)
df_methode_2_b = pd.DataFrame(docbase_b)
df_methode_2_b['MtSup'] = df_methode_2_b['MtSup'].astype(str)
df_methode_2_b['CaracSup'] = df_methode_2_b['CaracSup'].astype(str)
df_methode_2_b = df_methode_2_b.reset_index(drop=True)
df_methode_2_b.to_parquet(f'./fichiers/todo_parquet/{id_fichier_b}', engine='pyarrow')
 

Index(['Nature', 'ContNat', 'ArtSpe', 'CodRD', 'MtBudgPrec', 'MtPropNouv',
       'MtPrev', 'CredOuv', 'OpBudg', 'MtSup', 'CaracSup', 'DteStr',
       'LibellePoste', 'IdPost', 'LibelleColl', 'IdColl', 'NatCEPL',
       'Departement', 'Id_Fichier', 'TypOpBudg', 'MtRARPrec', 'LibCpte',
       'Operation', 'ContOp'],
      dtype='object')

0      {'@V': '24'}
1      {'@V': '24'}
2      {'@V': '24'}
3      {'@V': '24'}
4      {'@V': '24'}
           ...     
177    {'@V': '24'}
178    {'@V': '24'}
179    {'@V': '24'}
180    {'@V': '24'}
181    {'@V': '24'}
Name: Departement, Length: 182, dtype: object

In [64]:
df_methode_2_a.to_parquet(f'./fichiers/todo_parquet/{id_fichier_a}', engine='pyarrow')


'./fichiers/done_xml/612013.xml.gz'

'612013.xml.gz'